In [119]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [120]:
from helpers import load_csv_data

DATA_TRAIN_PATH = r'D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project1\data\train.csv'

y, X, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)
print(y.shape, X.shape)

(5000,) (5000, 30)


## Prepare cost and prediction functions

In [168]:
def hinge(Z):
    
    if type(Z) in {int, np.float64}:
        loss = max(0, 1-Z)
        
    if type(Z)==list:
        loss = []
        for z in Z:
            loss.append(max(0, 1-z))
            
    if type(Z)==np.ndarray:
        loss = []
        for z in list(Z):
            loss.append(max(0, 1-z))
        loss = np.array(loss).reshape(Z.shape)
        
    return loss

In [169]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    
    pri_obj = sum(hinge(np.multiply(y,X.dot(w)))) + (lambda_/2)*w.dot(w)
    
    return pri_obj

In [170]:
print(y.shape)
print(X.shape)

w = np.zeros(X.shape[1])

print(calculate_primal_objective(y, X, w, 0.1))

(5000,)
(5000, 30)
5000.0


In [171]:
from helpers import predict_labels

def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """

    # predict labels
    y_pred = predict_labels(w, X)
    
    # compute accuracy
    acc = len(np.where(np.array(y_pred) - np.array(y) == 0)[0])
    acc = acc / len(y)
    
    return acc

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [172]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    
    # get picked sample
    x_n, y_n = X[n,:], y[n]
    
    # compute SG: term with the sum
    if y_n*x_n.T.dot(w)>=0:
        sub_grad = 0
    else:
        sub_grad = -y_n*x_n

    # compute SG: term regularization
    SG = sub_grad + lambda_*w

    return SG

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [173]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 100000
    gamma = 1
    lambda_ = 0.01
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        
        # n = sample one data point uniformly at random data from X
        n = random.randint(0,num_examples-1)
        
        # compute the SG
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        
        # update w
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=5000.0
iteration=10000, cost=5000.0
iteration=20000, cost=5000.0
iteration=30000, cost=5000.0
iteration=40000, cost=5000.0
iteration=50000, cost=5000.0
iteration=60000, cost=5000.0
iteration=70000, cost=5000.0
iteration=80000, cost=5000.0
iteration=90000, cost=5000.0
training accuracy = 0.6714


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [ ]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    
    raise NotImplementedError
    return w, alpha

In [ ]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    raise NotImplementedError

In [ ]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 100000
    lambda_ = 0.01

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)